In [135]:
import pandas as pd

In [136]:
train = pd.read_csv('train.csv')

In [137]:
train.corrwith(train['Survived'])

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
dtype: float64

In [138]:
train.drop(['PassengerId', 'Pclass', 'Age', 'SibSp'], axis=1, inplace=True)

In [139]:
from pycaret.classification import *

In [140]:
s = setup(data=train, target='Survived', imputation_type='iterative', numeric_imputation='median',
categorical_imputation='mode', numeric_iterative_imputer='et', categorical_iterative_imputer='knn', silent=True,
normalize=True, transformation=True, train_size=0.95, session_id=2022)

,Description,Value
0,session_id,2022
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(891, 8)"
5,Missing Values,True
6,Numeric Features,1
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [141]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8120,0.0000,0.7364,0.7731,0.7534,0.6018,0.6032,0.0900
xgboost,Extreme Gradient Boosting,0.8039,0.8429,0.7061,0.7714,0.7365,0.5810,0.5831,2.9860
lda,Linear Discriminant Analysis,0.7979,0.8133,0.7576,0.7381,0.7453,0.5781,0.5808,1.1400
rf,Random Forest Classifier,0.7978,0.8550,0.6970,0.7656,0.7281,0.5680,0.5710,0.5960
lightgbm,Light Gradient Boosting Machine,0.7968,0.8525,0.7000,0.7577,0.7266,0.5657,0.5677,0.1640
dt,Decision Tree Classifier,0.7943,0.7768,0.6970,0.7594,0.7251,0.5615,0.5645,0.0960
et,Extra Trees Classifier,0.7908,0.8435,0.7061,0.7461,0.7243,0.5561,0.5578,0.8630
lr,Logistic Regression,0.7907,0.8526,0.7182,0.7394,0.7276,0.5579,0.5591,0.9170
svm,SVM - Linear Kernel,0.7896,0.0000,0.7303,0.7424,0.7248,0.5565,0.5653,0.0820
gbc,Gradient Boosting Classifier,0.7896,0.8453,0.6667,0.7659,0.7113,0.5472,0.5517,1.1090


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=2022,
                solver='auto', tol=0.001)

In [142]:
ridge = create_model('ridge')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8118,0.0000,0.7576,0.7576,0.7576,0.6037,0.6037
1,0.7882,0.0000,0.6970,0.7419,0.7188,0.5492,0.5499
2,0.8353,0.0000,0.7879,0.7879,0.7879,0.6533,0.6533
3,0.8235,0.0000,0.6970,0.8214,0.7541,0.6179,0.6230
4,0.8235,0.0000,0.7576,0.7812,0.7692,0.6264,0.6266
5,0.8118,0.0000,0.7273,0.7742,0.7500,0.5993,0.6000
6,0.8333,0.0000,0.7273,0.8276,0.7742,0.6430,0.6464
7,0.8214,0.0000,0.8182,0.7500,0.7826,0.6316,0.6333
8,0.7262,0.0000,0.6364,0.6562,0.6462,0.4229,0.4231


In [143]:
tunedmodel = tune_model(ridge, choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.0000,0.7576,0.7353,0.7463,0.5813,0.5815
1,0.8118,0.0000,0.7576,0.7576,0.7576,0.6037,0.6037
2,0.8353,0.0000,0.7879,0.7879,0.7879,0.6533,0.6533
3,0.8235,0.0000,0.6970,0.8214,0.7541,0.6179,0.6230
4,0.8353,0.0000,0.7879,0.7879,0.7879,0.6533,0.6533
5,0.8118,0.0000,0.7576,0.7576,0.7576,0.6037,0.6037
6,0.8452,0.0000,0.7576,0.8333,0.7937,0.6703,0.6722
7,0.8333,0.0000,0.8182,0.7714,0.7941,0.6543,0.6551
8,0.7262,0.0000,0.6364,0.6562,0.6462,0.4229,0.4231


In [144]:
test = pd.read_csv('cleantest.csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,B57 B59 B63 B66,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,B57 B59 B63 B66,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,B57 B59 B63 B66,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,27.0,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S
416,1308,3,"Ware, Mr. Frederick",male,27.0,0,0,359309,8.0500,B57 B59 B63 B66,S


In [145]:
pred = predict_model(tunedmodel, data=test)

In [146]:
pred

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Label
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,B57 B59 B63 B66,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,B57 B59 B63 B66,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,B57 B59 B63 B66,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,B57 B59 B63 B66,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,B57 B59 B63 B66,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,27.0,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S,0
416,1308,3,"Ware, Mr. Frederick",male,27.0,0,0,359309,8.0500,B57 B59 B63 B66,S,0


In [154]:
sub = pd.concat([pred['PassengerId'], pred['Label']], axis=1)

In [155]:
sub

,PassengerId,Label
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [157]:
sub = sub.rename(columns={'Label':'Survived'})

In [158]:
sub

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [159]:
submission = sub.to_csv('submission.csv', header=True, index=False)